# Point-to-Point Communication

Sending data from one point (process/task) to another point (process/task)

## send/recv

```julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

struct Point
  x::Float64
  y::Float64
end

p = Point(rank, rank)

if rank == 0
    MPI.send(p, comm; dest=1)
elseif rank > 0
    data = MPI.recv(comm; source=(rank - 1))
    MPI.send(p, comm; dest=(rank + 1) % size)
end

if rank == 0
    data = MPI.recv(comm; source=(size - 1))
end

print("My rank is $(rank)\n I received this: $(data)\n")
MPI.Barrier(comm)
```

In [1]:
%%file send_recv.jl
#!/usr/bin/env julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

struct Point
  x::Float64
  y::Float64
end

p = Point(rank, rank)

if rank == 0
    MPI.send(p, comm; dest=1)
elseif rank > 0
    data = MPI.recv(comm; source=(rank - 1))
    MPI.send(p, comm; dest=(rank + 1) % size)
end

if rank == 0
    data = MPI.recv(comm; source=(size - 1))
end

print("My rank is $(rank)\n I received this: $(data)\n")
MPI.Barrier(comm)

Writing send_recv.jl


If the preceding code is saved as a file [send_recv.jl](send_recv.jl), we can run

In [2]:
!~/.julia/bin/mpiexecjl --project=.. -np 3 julia send_recv.jl

My rank is 1
 I received this: Point(0.0, 0.0)


My rank is 2
 I received this: Point(1.0, 1.0)


My rank is 0
 I received this: Point(2.0, 2.0)


## Send/Recv

```julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)


send_buf = Array{Float64}(undef, 10)
recv_buf = Array{Float64}(undef, 10)

fill!(send_buf, Float64(rank))

if rank == 0
    MPI.Send(send_buf, comm; dest=1)
elseif rank > 0
    MPI.Recv!(recv_buf, comm; source=(rank - 1))
    MPI.Send(send_buf, comm; dest=(rank + 1) % size)
end

if rank == 0
    MPI.Recv!(recv_buf, comm; source=(size - 1))
end

print("My rank is $(rank)\n I received this: $(recv_buf)\n")
MPI.Barrier(comm)
```

In [3]:
%%file Send_Recv.jl
#!/usr/bin/env julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)


send_buf = Array{Float64}(undef, 10)
recv_buf = Array{Float64}(undef, 10)

fill!(send_buf, Float64(rank))

if rank == 0
    MPI.Send(send_buf, comm; dest=1)
elseif rank > 0
    MPI.Recv!(recv_buf, comm; source=(rank - 1))
    MPI.Send(send_buf, comm; dest=(rank + 1) % size)
end

if rank == 0
    MPI.Recv!(recv_buf, comm; source=(size - 1))
end

print("My rank is $(rank)\n I received this: $(recv_buf)\n")
MPI.Barrier(comm)

Writing Send_Recv.jl


If the preceding code is saved as a file [Send_Recv.jl](Send_Recv.jl), we can run

In [4]:
!~/.julia/bin/mpiexecjl -np 3 julia Send_Recv.jl

ERROR: 

ArgumentError: Package MPI not found in current path:
- Run `import Pkg; Pkg.add("MPI")` to install the MPI package.

Stacktrace:
 [1] require(into::Module, mod::Symbol)
   @ Base ./loading.jl:967


## Isend/Irecv

```julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)


send_buf = Array{Float64}(undef, 10)
recv_buf = Array{Float64}(undef, 10)

fill!(send_buf, Float64(rank))

if rank == 0
    send_status = MPI.Isend(send_buf, comm; dest=1)
elseif rank > 0
    recv_status = MPI.Irecv!(recv_buf, comm; source=(rank - 1))
    send_status = MPI.Isend(send_buf, comm; dest=(rank + 1) % size)
end

if rank == 0
    recv_status = MPI.Irecv!(recv_buf, comm; source=(size - 1))
end

stats = MPI.Waitall!([send_status, recv_status])

print("My rank is $(rank)\n I received this: $(recv_buf)\n")
```

In [5]:
%%file Isend_Irecv.jl
#!/usr/bin/env julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)


send_buf = Array{Float64}(undef, 10)
recv_buf = Array{Float64}(undef, 10)

fill!(send_buf, Float64(rank))

if rank == 0
    send_status = MPI.Isend(send_buf, comm; dest=1)
elseif rank > 0
    recv_status = MPI.Irecv!(recv_buf, comm; source=(rank - 1))
    send_status = MPI.Isend(send_buf, comm; dest=(rank + 1) % size)
end

if rank == 0
    recv_status = MPI.Irecv!(recv_buf, comm; source=(size - 1))
end

stats = MPI.Waitall!([send_status, recv_status])

print("My rank is $(rank)\n I received this: $(recv_buf)\n")

Writing Isend_Irecv.jl


If the preceding code is saved as a file `Isend_Irecv.jl`, we can run

In [6]:
!~/.julia/bin/mpiexecjl -np 3 julia ISend_IRecv.jl

ERROR: 

ArgumentError: Package MPI not found in current path:
- Run `import Pkg; Pkg.add("MPI")` to install the MPI package.

Stacktrace:
 [1] require(into::Module, mod::Symbol)
   @ Base ./loading.jl:967


In [7]:
%%javascript
function hideElements(elements, start) {
    for(var i = 0, length = elements.length; i < length;i++) {
        if(i >= start) {
            elements[i].style.display = "none";
        }
    }
}

var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)

<IPython.core.display.Javascript object>